In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
# patching
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

from utils.utils import find_nearest

In [2]:
path = Path(".").resolve().parent.parent / "data" / "sensors_characteristics"

In [3]:
path

WindowsPath('C:/Users/fmark/PycharmProjects/dissertation/data/sensors_characteristics')

In [4]:
reference_voltage = 3.3
steps = 2**12

In [5]:
steps

4096

In [6]:
step = reference_voltage / steps

In [7]:
step

0.0008056640625

In [8]:
codes = [i for i in range(steps)]
voltages = [(code / steps) * reference_voltage + step for code in codes]
df_mcp_4725 = pd.DataFrame({"code": codes, "voltage": voltages})

In [9]:
df_mcp_4725.head()

,code,voltage
0,0,0.000806
1,1,0.001611
2,2,0.002417
3,3,0.003223
4,4,0.004028


In [10]:
df_mcp_4725["divided_voltage"] = df_mcp_4725["voltage"] / 100

In [11]:
df_mcp_4725["divided_voltage (mV)"] = df_mcp_4725["divided_voltage"] * 1000

In [12]:
df_mcp_4725

,code,voltage,divided_voltage,divided_voltage (mV)
0,0,0.000806,0.000008,0.008057
1,1,0.001611,0.000016,0.016113
2,2,0.002417,0.000024,0.024170
3,3,0.003223,0.000032,0.032227
4,4,0.004028,0.000040,0.040283
...,...,...,...,...
4091,4091,3.296777,0.032968,32.967773
4092,4092,3.297583,0.032976,32.975830
4093,4093,3.298389,0.032984,32.983887
4094,4094,3.299194,0.032992,32.991943


In [13]:
sensor_characteristics = pd.read_csv(path / "ТЖК (J).csv",index_col="temp")

In [15]:
sensor_characteristics.head()

,value
temp,
-210,-8.095
-209,-8.076
-208,-8.057
-207,-8.037
-206,-8.017


In [17]:
df = pd.DataFrame()

In [18]:
df

""


In [20]:
min_temp, max_temp = -50, 100

In [21]:
min_value = sensor_characteristics.loc[min_temp]["value"]
max_value = sensor_characteristics.loc[max_temp]["value"]

In [22]:
min_value

-2.431

In [23]:
max_value

5.269

In [24]:
df["voltage"] = sensor_characteristics[sensor_characteristics.index.isin(range(min_temp, max_temp + 1))]["value"]

In [25]:
df.head(10)

,voltage
temp,
-50,-2.431
-49,-2.385
-48,-2.338
-47,-2.291
-46,-2.244
-45,-2.197
-44,-2.150
-43,-2.103
-42,-2.055


In [26]:
_simulation_range = [value for value in df_mcp_4725["divided_voltage (mV)"] if min_value < value < max_value]

In [28]:
_simulation_range[0:10]

[0.008056640625,
 0.01611328125,
 0.024169921874999997,
 0.0322265625,
 0.040283203125,
 0.04833984375,
 0.05639648437499999,
 0.064453125,
 0.072509765625,
 0.08056640625]

In [32]:
df["calc"] = df["voltage"].apply(lambda x: round(math.copysign(1, x) * find_nearest(_simulation_range, abs(x)), 5))

In [34]:
df.head()

,voltage,calc
temp,,
-50,-2.431,-2.43311
-49,-2.385,-2.38477
-48,-2.338,-2.33643
-47,-2.291,-2.28809
-46,-2.244,-2.24780


In [35]:
df["code"] = df_mcp_4725["v"].apply(lambda x: df_mcp_4725[df_mcp_4725["divided_voltage (mV)"] == df.iloc[x]["calc"]]["code"], axis=0)

KeyError: 'te'

In [55]:
df["calc"].apply(lambda x: int(df_mcp_4725[df_mcp_4725["divided_voltage (mV)"] == x].iloc[0]["code"]))

temp
0        0
1        5
2       12
3       18
4       24
      ... 
96     626
97     633
98     639
99     646
100    652
Name: calc, Length: 101, dtype: int64

In [52]:
df_mcp_4725[df_mcp_4725["divided_voltage (mV)"] == df["calc"][100]]["code"]

652    652
Name: code, dtype: int64

In [51]:
df["calc"].apply(lambda x: x)

temp
0      0.008057
1      0.048340
2      0.104736
3      0.153076
4      0.201416
         ...   
96     5.051514
97     5.107910
98     5.156250
99     5.212646
100    5.260986
Name: calc, Length: 101, dtype: float64